In [12]:
# Importation des bibliothèques nécessaires
import os
import pandas as pd
import numpy as np
from openpyxl import load_workbook, Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font
import json
from typing import List, Optional


Import de la dernière base d'oeufs stockée dans data (extraite du fichier parquet) avec résultat de l'analyse : breeding, quantity, french +résultat de l'OCR

In [13]:
df_oeufs_fr = pd.read_csv('C:\\dev\\git\\13_empreinte_souffrance\\backend\\app\\scripts\\data\\processed_products.csv')
df_oeufs_fr

,code,categories_tags,labels_tags,product_name,generic_name,quantity,product_quantity_unit,product_quantity,allergens_tags,ingredients_tags,...,images,texte_ocr,breeding_type_related,weight_related,proba_1,proba_2,proba_3,egg_count,caliber,breeding
0,00003100,"['en:farming-products', 'en:eggs']",[],"[{'lang': 'main', 'text': 'Hard Boiled Eggs'},...",[],2,NaN,0.000000,['en:eggs'],"['fr:eggs', 'en:e330', 'fr:sodium-benzoate', '...",...,"[{'key': 'front', 'imgid': 1, 'rev': 3, 'sizes...",10 cadrojac . fr . (31.091.010) . ce . pc10 . ...,NaN,"2 ea, 0.792kg",processed meat,meat,sweetened beverages,2,no caliber,no breeding
1,0011110797698,"['en:farming-products', 'en:eggs', 'en:undefin...",NaN,"[{'lang': 'main', 'text': 'Natural Grade Aa La...",[],50 g,g,50.000000,[],['en:large-brown-eggs'],...,"[{'key': '1', 'imgid': None, 'rev': None, 'siz...",sell by . 100 mg omega 3 fatty acids per egg ....,"Natural Cage Free, Grain Fed",12 EGGS NET WT 24 OZ (1 LB 8 02) 681g,biscuits and cakes,eggs,one dish meals,1,no caliber,no breeding
2,0011110806543,"['en:farming-products', 'en:eggs']",NaN,"[{'lang': 'main', 'text': '100% Egg Whites'}, ...",[],NaN,NaN,NaN,['en:eggs'],"['en:egg-white', 'en:egg']",...,[],NaN,NaN,NaN,NaN,NaN,NaN,-1,no caliber,no breeding
3,0011110828897,"['en:farming-products', 'en:eggs']",NaN,"[{'lang': 'main', 'text': 'Kroger, break-free,...",[],NaN,NaN,NaN,['en:eggs'],"['en:egg-white', 'en:egg', 'en:contains-1-and-...",...,[],NaN,NaN,NaN,NaN,NaN,NaN,-1,no caliber,no breeding
4,0011110846037,"['en:farming-products', 'en:eggs']",NaN,"[{'lang': 'main', 'text': '100% Liquid Egg Whi...",[],NaN,NaN,NaN,['en:eggs'],"['en:liquid-egg-white', 'en:egg', 'en:egg-white']",...,[],NaN,NaN,NaN,NaN,NaN,NaN,-1,no caliber,no breeding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7947,00303576,"['en:farming-products', 'en:eggs', 'en:chicken...",NaN,[],[],NaN,NaN,NaN,[],NaN,...,"[{'key': '1', 'imgid': None, 'rev': None, 'siz...",NaN,NaN,NaN,NaN,NaN,NaN,-1,no caliber,no breeding
7948,38006784,"['en:farming-products', 'en:eggs', 'en:chicken...",NaN,"[{'lang': 'main', 'text': 'Кокоши яйца'}, {'la...",[],10 бр.,NaN,0.000000,['en:eggs'],"['en:chicken-egg', 'en:egg']",...,"[{'key': '1', 'imgid': None, 'rev': None, 'siz...",NaN,NaN,NaN,NaN,NaN,NaN,10,no caliber,barn
7949,5905784305233,"['en:farming-products', 'en:eggs']",['en:nutriscore'],"[{'lang': 'main', 'text': 'Eggs'}, {'lang': 'e...",[],NaN,NaN,NaN,[],NaN,...,"[{'key': '1', 'imgid': None, 'rev': None, 'siz...",NaN,NaN,NaN,NaN,NaN,NaN,-1,no caliber,no breeding
7950,4770190117678,"['en:farming-products', 'en:eggs', 'en:chicken...","['en:lt-nkp-001', 'en:nacionalinė-kokybė']","[{'lang': 'main', 'text': 'HAPPY Deep Litter C...",[],530 g,g,530.000000,['en:eggs'],['en:egg'],...,[],NaN,NaN,NaN,NaN,NaN,NaN,9,no caliber,no breeding


Définition des listes des codes produits à recenser sur l'excel :

- ceux dont élevage et quantité manquent
- ceux dont élevage manque
- ceux qui ne sont pas plein air et dont quantité manque


In [14]:
# Filter products with no breeding type and no quantity
dfoeufs_fr_no_breeding_no_quantity = df_oeufs_fr.loc[
    (df_oeufs_fr["breeding"].isin(["not managed", "no breeding"])) &
    (df_oeufs_fr["egg_count"] < 0), 
    'code'
]

# Filter products with no breeding type but with quantity
dfoeufs_fr_no_breeding = df_oeufs_fr.loc[
    (df_oeufs_fr["breeding"].isin(["not managed", "no breeding"])) &
    (df_oeufs_fr["egg_count"] >= 0), 
    'code'
]

# Filter products with breeding type (not free range) but no quantity
dfoeufs_fr_no_quantity_not_free_range = df_oeufs_fr.loc[
    (df_oeufs_fr["breeding"] != "free_range") &
    (df_oeufs_fr["breeding"] != "not managed") & 
    (df_oeufs_fr["breeding"] != "no breeding") & 
    (df_oeufs_fr["egg_count"] < 0), 
    'code'
]

# Filter products with breeding type but no quantity (non-utilisé)
dfoeufs_fr_no_quantity = df_oeufs_fr.loc[
    (df_oeufs_fr["breeding"] != "not managed") & 
    (df_oeufs_fr["breeding"] != "no breeding") & 
    (df_oeufs_fr["egg_count"] == 0), 
    'code'
]

# Convert to lists
no_breeding_no_quantity = dfoeufs_fr_no_breeding_no_quantity.tolist()
no_breeding = dfoeufs_fr_no_breeding.tolist()
no_quantity = dfoeufs_fr_no_quantity.tolist()
no_quantity_not_free_range = dfoeufs_fr_no_quantity_not_free_range.tolist()

In [15]:


class ExcelProductGenerator:
    """Handles generation of product Excel files from DataFrame data."""
    
    # Class constants for better maintainability
    DEFAULT_SHEET_NAME = "Feuille1"
    EXCEL_EXTENSION = ".xlsx"
    IMAGE_COUNT = 10
    
    BASE_COLUMNS = ["Traité", "Info invisible", "Pas œuf", "Code et URL produit", "Autres"]
    OPTIONAL_COLUMNS = {
        "breeding": "Elevage",
        "quantity": ["Qté", "Calibre"]
    }
    
    # Nouvelles colonnes à ajouter avant les images
    NEW_COLUMNS = ["OCR", "breeding_type_related", "weight_type_related", "predictions_categories"]
    
    COLUMN_RENAMES = {
        "Traité": "Traité",
        "Info invisible": "Info invisible", 
        "Pas œuf": "Pas œuf / comment.",
        "Code et URL produit": "Code et URL produit",
        "Image front": "Image principale",
        "Image ingredients": "Image ingrédients",
        "Autres": "Nom, ingrédients, labels, catégories",
        "Elevage": "Elevage\n (cage, sol, plein-air, bio ou 3, 2, 1, 0)",
        "Qté": "Quantité\n(nb d'œufs ou poids)",
        "Calibre": "Calibre\n(S, M, L, XL ou petit, moyen, gros, très gros)",
        "OCR": "OCR",
        "breeding_type_related": "breeding_type_related",
        "weight_type_related": "weight_type_related",
        "predictions_categories": "predictions categories"
    }

    # Column widths for formatting
    COLUMN_WIDTHS = {
        "Traité": 8,
        "Info invisible": 10,
        "Pas œuf / comment.": 10,
        "Code et URL produit": 15,
        "Nom, ingrédients, labels, catégories": 22,
        "Image principale": 50,
        "Image ingrédients": 50,
        "Elevage\n (cage, sol, plein-air, bio ou 3, 2, 1, 0)": 15,
        "Quantité\n(nb d'œufs ou poids)": 13,
        "Calibre\n(S, M, L, XL ou petit, moyen, gros, très gros)": 17,
        "OCR": 50,
        "breeding_type_related": 15,
        "weight_type_related": 15,
        "predictions categories": 15,
        **{f"Image {i}": 50 for i in range(1, 11)}
    }

    def __init__(self, data_path: str = "../data/"):
        """Initialize the generator with data path."""
        self.data_path = data_path

    def build_row_from_df(self, product: dict, code: str, df: pd.DataFrame, 
                         show_breeding: bool, show_quantity: bool,
                         breeding_value: Optional[str] = None,
                         quantity_value: Optional[str] = None) -> dict:
        """Build a complete row from product data and DataFrame."""
        
        # Extract product information
        product_name_raw = product.get("product_name", [])
        generic_name_raw = product.get("generic_name", "")
        ingredients = product.get("ingredients_tags", [])
        labels = product.get("labels_tags", [])
        categories = product.get("categories_tags", [])
        images_raw = product.get("images", [])

        # Parse images
        if isinstance(images_raw, str):
            try:
                images_list = json.loads(images_raw.replace("'", '"').replace("None", "null"))
            except json.JSONDecodeError:
                images_list = []
        else:
            images_list = images_raw if images_raw else []

        # Parse product names
        if isinstance(product_name_raw, str):
            try:
                product_name_list = json.loads(product_name_raw.replace("'", '"'))
            except json.JSONDecodeError:
                product_name_list = []
        else:
            product_name_list = product_name_raw

        if isinstance(product_name_list, list):
            product_name = '\n'.join(
                d.get('text', '') 
                for d in product_name_list 
                if isinstance(d, dict) and d.get('lang') != "main"
            )
        else:
            product_name = str(product_name_list)

        # Parse generic names
        if isinstance(generic_name_raw, str):
            try:
                generic_name_list = json.loads(generic_name_raw.replace("'", '"'))
            except json.JSONDecodeError:
                generic_name_list = []
        else:
            generic_name_list = generic_name_raw

        if isinstance(generic_name_list, list):
            generic_name = '\n'.join(
                d.get('text', '') 
                for d in generic_name_list 
                if isinstance(d, dict) and d.get('lang') != "main"
            )
        else:
            generic_name = str(generic_name_list)

        # Format product code and build image URLs
        code_padded = str(code).zfill(13)
        path = f"{code_padded[:3]}/{code_padded[3:6]}/{code_padded[6:9]}/{code_padded[9:]}"

        # Get image keys (only integer keys)
        image_keys = sorted([
            img["key"] for img in images_list
            if isinstance(img, dict) and str(img.get("key")).isdigit()
        ], key=int)

        # Build image URLs
        image_urls = [
            f"https://images.openfoodfacts.net/images/products/{path}/{k}.jpg"
            for k in image_keys
        ][:self.IMAGE_COUNT]

        # Build base row
        row = {
            "Traité": "",
            "Info invisible": "",
            "Pas œuf": "",
            "Code et URL produit": code,
            "Autres": f"{product_name}\n{generic_name}\n{ingredients}\n{labels}\n{categories}",
            **{
                f"Image {i+1}": f'=IMAGE("{image_urls[i]}")' if i < len(image_urls) else ""
                for i in range(self.IMAGE_COUNT)
            }
        }

        # Add optional columns
        if show_breeding:
            row["Elevage"] = breeding_value if breeding_value is not None else ""
        if show_quantity:
            row["Qté"] = quantity_value if quantity_value is not None else ""
            row["Calibre"] = ""

        # Add new columns values
        new_values = self._get_new_column_values(code, df)
        row.update(new_values)

        return row

    def format_excel_file(self, filename: str, columns: list[str], product_codes: list[str], sheet_name: str = "Feuille1"):
        """Apply formatting to the Excel file."""
        wb = load_workbook(filename)
        ws = wb[sheet_name]

        # Add hyperlink to product page
        code_col_idx = list(columns).index("Code et URL produit") + 1
        for i, code in enumerate(product_codes):
            cell = ws.cell(row=i + 2, column=code_col_idx)
            cell.hyperlink = f"https://fr.openfoodfacts.org/produit/{code}"
            cell.style = "Hyperlink"

        # Apply column widths
        for idx, name in enumerate(columns, 1):
            ws.column_dimensions[get_column_letter(idx)].width = self.COLUMN_WIDTHS.get(name, 15)

        # Set row heights (for images)
        for r in range(2, 2 + len(product_codes)):
            ws.row_dimensions[r].height = 300

        # Define styles
        fill_gray = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
        border = Border(
            left=Side(style="dotted", color="E0E0E0"),
            right=Side(style="dotted", color="E0E0E0"),
            top=Side(style="thin", color="AAAAAA"),
            bottom=Side(style="thin", color="AAAAAA")
        )

        # Apply alignment and borders to all cells
        for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.alignment = Alignment(wrap_text=True, vertical="center", horizontal="center")
                cell.border = border

        # Format header row
        for cell in ws[1]:
            cell.alignment = Alignment(horizontal="center", wrap_text=True, vertical="center")
            cell.font = Font(size=8, bold=True)

        # Freeze panes
        if "Pas œuf / comment." in columns:
            freeze_col_idx = list(columns).index("Pas œuf / comment.") + 2
            ws.freeze_panes = ws.cell(row=2, column=freeze_col_idx)

            # Apply gray fill to frozen columns
            for r in range(2, ws.max_row + 1):
                for c in range(freeze_col_idx, ws.max_column + 1):
                    ws.cell(row=r, column=c).fill = fill_gray

        wb.save(filename)
        print(f"✅ Feuille Excel '{sheet_name}' générée dans {filename}")

    def generate_excel(self, 
                      product_codes: List[str],
                      df: pd.DataFrame,
                      output_filename: str,
                      show_col_breeding: bool = True,
                      show_cols_quantity: bool = True,
                      fetch_breeding: bool = False,
                      fetch_quantity: bool = False,
                      sheet_name: str = DEFAULT_SHEET_NAME) -> str:
        """
        Generate Excel file from product codes and DataFrame.
        
        Args:
            product_codes: List of product codes to process
            df: Source DataFrame containing product data
            output_filename: Output Excel filename
            show_col_breeding: Whether to include breeding column
            show_cols_quantity: Whether to include quantity columns
            fetch_breeding: Whether to fetch breeding data from 'breeding' column
            fetch_quantity: Whether to fetch quantity data from 'w_eggs' column
            sheet_name: Name of the Excel sheet
            
        Returns:
            Full path to the created Excel file
        """
        file_path = self._prepare_file_path(output_filename)
        
        # Process products and build rows
        rows, valid_codes = self._process_products(
            product_codes, df, show_col_breeding, show_cols_quantity, 
            fetch_breeding, fetch_quantity
        )
        
        if not rows:
            raise ValueError("No valid products found to process")
        
        # Create DataFrame with proper column structure
        df_output = self._create_output_dataframe(
            rows, show_col_breeding, show_cols_quantity
        )
        
        # Write to Excel
        self._write_to_excel(df_output, file_path, sheet_name)
        
        # Apply formatting
        self.format_excel_file(file_path, df_output.columns.tolist(), valid_codes, sheet_name)
        
        return file_path

    def _prepare_file_path(self, filename: str) -> str:
        """Ensure filename has correct extension and build full path."""
        if not filename.endswith(self.EXCEL_EXTENSION):
            filename += self.EXCEL_EXTENSION
        return os.path.join(self.data_path, filename)

    def _process_products(self, 
                         product_codes: List[str], 
                         df: pd.DataFrame,
                         show_breeding: bool,
                         show_quantity: bool,
                         fetch_breeding: bool,
                         fetch_quantity: bool) -> tuple[List[dict], List[str]]:
        """Process product codes and return rows and valid codes."""
        rows = []
        valid_codes = []
        
        for code in product_codes:
            try:
                product = self._fetch_product_data(code, df)
                if product:
                    # Fetch additional data if requested
                    breeding_value = None
                    quantity_value = None
                    
                    if fetch_breeding and 'breeding' in df.columns:
                        breeding_value = self._get_value_from_df(code, df, 'breeding')
                    
                    if fetch_quantity and 'w_eggs' in df.columns:
                        quantity_value = self._get_value_from_df(code, df, 'w_eggs')
                    
                    row = self.build_row_from_df(
                        product, code, df, show_breeding, show_quantity,
                        breeding_value, quantity_value
                    )
                    rows.append(row)
                    valid_codes.append(code)
            except Exception as e:
                print(f"Warning: Failed to process product {code}: {e}")
                continue
                
        return rows, valid_codes

    def _get_value_from_df(self, code: str, df: pd.DataFrame, column: str) -> Optional[str]:
        """Get value from DataFrame for a specific product code and column."""
        try:
            mask = df['code'] == code
            if mask.any():
                value = df.loc[mask, column].iloc[0]
                return str(value) if pd.notna(value) else None
        except (KeyError, IndexError):
            pass
        return None

    def _get_new_column_values(self, code: str, df: pd.DataFrame) -> dict:
        """Get values for the new columns from DataFrame."""
        values = {}
        
        # OCR from 'texte_ocr' column
        values['OCR'] = self._get_value_from_df(code, df, 'texte_ocr') or ""
        
        # breeding_type_related and weight_type_related
        values['breeding_type_related'] = self._get_value_from_df(code, df, 'breeding_type_related') or ""
        values['weight_type_related'] = self._get_value_from_df(code, df, 'weight_type_related') or ""
        
        # predictions categories - concatenate proba_1, proba_2, proba_3 with \n
        proba_1 = self._get_value_from_df(code, df, 'proba_1') or ""
        proba_2 = self._get_value_from_df(code, df, 'proba_2') or ""
        proba_3 = self._get_value_from_df(code, df, 'proba_3') or ""
        
        # Filter out empty values and join with \n
        proba_values = [p for p in [proba_1, proba_2, proba_3] if p]
        values['predictions_categories'] = "\n".join(proba_values)
        
        return values

    def _fetch_product_data(self, code: str, df: pd.DataFrame) -> Optional[dict]:
        """Fetch product data from DataFrame"""
        row = df[df['code'] == code]
        if not row.empty:
            return row.iloc[0].replace({np.nan: None}).to_dict()
        else:
            return {}

    def _create_column_order(self, show_breeding: bool, show_quantity: bool) -> List[str]:
        """Create the column order based on display options."""
        columns = ["Traité"]
        
        if show_breeding:
            columns.append(self.OPTIONAL_COLUMNS["breeding"])
            
        if show_quantity:
            columns.extend(self.OPTIONAL_COLUMNS["quantity"])
            
        columns.extend(self.BASE_COLUMNS[1:])  # Skip "Traité" as it's already added
        
        # Add new columns before images
        columns.extend(self.NEW_COLUMNS)
        
        # Add image columns
        columns.extend([f"Image {i+1}" for i in range(self.IMAGE_COUNT)])
        
        return columns

    def _create_output_dataframe(self, 
                               rows: List[dict],
                               show_breeding: bool,
                               show_quantity: bool) -> pd.DataFrame:
        """Create and format the output DataFrame."""
        column_order = self._create_column_order(show_breeding, show_quantity)
        
        df_output = pd.DataFrame(rows)
        df_output = df_output.reindex(columns=column_order)
        df_output.rename(columns=self.COLUMN_RENAMES, inplace=True)
        
        return df_output

    def _write_to_excel(self, 
                       df_output: pd.DataFrame,
                       file_path: str,
                       sheet_name: str) -> None:
        """Write DataFrame to Excel file."""
        # Handle existing file
        if os.path.exists(file_path):
            wb = load_workbook(file_path)
            if sheet_name in wb.sheetnames:
                del wb[sheet_name]
            ws = wb.create_sheet(title=sheet_name)
        else:
            wb = Workbook()
            ws = wb.active
            ws.title = sheet_name

        # Write headers
        for col_idx, col_name in enumerate(df_output.columns, start=1):
            ws.cell(row=1, column=col_idx, value=col_name)

        # Write data
        for row_idx, row_data in enumerate(df_output.itertuples(index=False), start=2):
            for col_idx, cell_value in enumerate(row_data, start=1):
                ws.cell(row=row_idx, column=col_idx, value=cell_value)

        wb.save(file_path)


# Backward compatibility function
def generate_product_excel_from_df(product_codes: List[str],
                                   df: pd.DataFrame,
                                   output_excel_filename: str,
                                   show_col_breeding: bool = True,
                                   show_cols_quantity: bool = True,
                                   fetch_breeding: bool = False,
                                   fetch_quantity: bool = False,
                                   data_path: str = "../data/",
                                   sheet_name: str = "Feuille1") -> str:
    """
    Backward compatibility wrapper for the refactored class-based implementation.
    
    This function maintains the same interface as the original while using
    the new ExcelProductGenerator class internally.
    
    Args:
        product_codes: List of product codes to process
        df: Source DataFrame containing product data
        output_excel_filename: Output Excel filename
        show_col_breeding: Whether to include breeding column
        show_cols_quantity: Whether to include quantity columns
        fetch_breeding: Whether to fetch breeding data from 'breeding' column
        fetch_quantity: Whether to fetch quantity data from 'w_eggs' column
        data_path: Path to data directory
        sheet_name: Name of the Excel sheet
        
    Returns:
        Full path to the created Excel file
    """
    generator = ExcelProductGenerator(data_path)
    return generator.generate_excel(
        product_codes=product_codes,
        df=df,
        output_filename=output_excel_filename,
        show_col_breeding=show_col_breeding,
        show_cols_quantity=show_cols_quantity,
        fetch_breeding=fetch_breeding,
        fetch_quantity=fetch_quantity,
        sheet_name=sheet_name
    )

In [17]:
# Afficher les produits contenant des ingredients de type "barn"
confirm = input("Voulez-vous créer l'excel de test ? (o/n): ")

if confirm.lower() == 'o':
    print("Création d'un excel de test pour les produits 00257961, 00388500, 0055000002240")
    sample_codes = ["00257961", "00388500", "0055000002240", "00003100"]
    generate_product_excel_from_df(df = df_oeufs_fr,
        product_codes=sample_codes,
        output_excel_filename="products_test.xlsx",
        show_col_breeding=True,
        show_cols_quantity=True,
        sheet_name="Test produits",
        fetch_breeding=True,
        fetch_quantity=True,
    )
    os.startfile("..\\data\\products_test.xlsx")

else:
    print("Opération passée")

Création d'un excel de test pour les produits 00257961, 00388500, 0055000002240
✅ Feuille Excel 'Test produits' générée dans ../data/products_test.xlsx


In [ ]:
# Afficher les produits contenant des ingredients de type "barn"
confirm = input("Voulez-vous créer l'excel pour tous les produits ? (o/n): ")

if confirm.lower() == 'o':
    generate_product_excel_from_df(df = df_oeufs_fr,
        product_codes=no_breeding_no_quantity,
        output_excel_filename="products_to_tag_14_06_25.xlsx",
        show_col_breeding=True,
        show_cols_quantity=True,
        fetch_breeding=False,
        fetch_quantity=False,

        sheet_name="no_breeding_no_quantity"
    )

    generate_product_excel_from_df(df = df_oeufs_fr,
        product_codes=no_breeding,
        output_excel_filename="products_to_tag_14_06_25.xlsx",
        show_col_breeding=True,
        show_cols_quantity=True,
        sheet_name="no_breeding",
        fetch_breeding=False,
        fetch_quantity=True,
    )

    generate_product_excel_from_df(df = df_oeufs_fr,
        product_codes=no_quantity_not_free_range,
        output_excel_filename="products_to_tag_14_06_25.xlsx",
        show_col_breeding=True,
        show_cols_quantity=True,
        fetch_breeding=True,
        fetch_quantity=False,
        sheet_name="no_quantity_not_free_range"
    )
    os.startfile("..\\data\\products_to_tag_14_06_25.xlsx")

else:
    print("Opération passée")


✅ Feuille Excel 'no_breeding_no_quantity' générée dans ../data/products_to_tag_14_06_25.xlsx
✅ Feuille Excel 'no_breeding' générée dans ../data/products_to_tag_14_06_25.xlsx
✅ Feuille Excel 'no_quantity_not_free_range' générée dans ../data/products_to_tag_14_06_25.xlsx
